# Assignment #4 - automation
## Advanced Databases
### Kristjan Lõhmus, Rimmo Rõõm


### Preliminaries

Install postgres connector and establish connection

In [22]:
import psycopg2
import pandas.io.sql as sqlio
import pandas as pd

In [23]:
conn = psycopg2.connect(
    host= 'localhost',
    password = "postgres",
    user = "postgres",
    port = 5432,
    )
conn.autocommit = True
cursor = conn.cursor()
cursor.execute('CREATE OR REPLACE SCHEMA sales_info;')
cursor.execute('set search_path = "sales_info";')

### A. Constraints programming

#### 1. Implement the structure of this system by creating the tables.
![Schema to implement](screenshots/schema.png)

In [24]:
cursor.execute("""
CREATE TABLE Invoice (
                         IdInv SERIAL PRIMARY KEY,
                         DateInv DATE NOT NULL,
                         DatePay DATE,
                         ModePay VARCHAR(50) DEFAULT 'Card' CHECK (ModePay IN ('Check', 'Card', 'Cash')),
                         Status VARCHAR(50) DEFAULT 'Not Paid' CHECK (Status IN ('Paid', 'Not Paid'))
);

-- Create Product table
CREATE TABLE Product (
                         BareCode VARCHAR(50) PRIMARY KEY,
                         Label VARCHAR(100) NOT NULL,
                         Category VARCHAR(50),
                         Stock INT CHECK (Stock > 0)
);

-- Create Customer table
CREATE TABLE Customer (
                          IdCust SERIAL PRIMARY KEY,
                          Name VARCHAR(100) NOT NULL,
                          Phone VARCHAR(20) NOT NULL CHECK (Phone LIKE '00%')
);

-- Create OrderC table
CREATE TABLE OrderC (
                        IdOrd SERIAL PRIMARY KEY,
                        DateOrd DATE NOT NULL,
                        StatusOrd VARCHAR(50) DEFAULT 'Pending' CHECK (StatusOrd IN ('validate', 'Pending', 'Partial')),
                        IdCust INT REFERENCES Customer(IdCust),
                        IdInv INT REFERENCES Invoice(IdInv)
);

-- Create Delivery table
CREATE TABLE Delivery (
                          IdDel SERIAL PRIMARY KEY,
                          DateExp DATE,
                          DateDel DATE
);

-- Create OrderLigne table
CREATE TABLE OrderLigne (
                            IdOrLi SERIAL PRIMARY KEY,
                            Quantity INT NOT NULL CHECK (Quantity > 0),
                            StatusOrLig VARCHAR(50) DEFAULT 'Preparation' CHECK (StatusOrLig IN ('Preparation', 'prepared', 'Supply')),
                            IdOrd INT REFERENCES OrderC(IdOrd),
                            IdDel INT REFERENCES Delivery(IdDel) NULL,
                            BareCode VARCHAR(50) REFERENCES Product(BareCode)
);
""")

Resulting schema:

![Implemented schema](screenshots/01.sales_info.png)

#### 2. Data insertion

##### a. Let’s assume we have customer (‘Alice’,’002736014251’); and four Products (AAA001, ‘Wood Chair’, ‘House’, 10); (AAA002, ‘Iron bed’, ‘House’, 5); (AAA003, ‘Mathematics Book’, ‘Education’, 2); (AAA004, ‘Couch’, ‘House’, 2) in our system.

In [25]:
cursor.execute("""INSERT INTO customer(Name, Phone) values ('Alice','002736014251');""")
sqlio.read_sql_query("SELECT * FROM customer", conn).head()

C:\Users\KristjanLõhmus\AppData\Local\Temp\ipykernel_32036\1667529848.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sqlio.read_sql_query("SELECT * FROM customer", conn).head()


,idcust,name,phone
0,1,Alice,002736014251


In [26]:
cursor.execute("""
insert into product(BareCode, Label, Category, Stock) values
('AAA001', 'Wood Chair', 'House', 10),
('AAA002', 'Iron bed', 'House', 5),
('AAA003', 'Mathematics Book', 'Education', 2),
('AAA004', 'Couch', 'House', 2);
""")
initial_stock = sqlio.read_sql_query("SELECT * FROM product", conn)
initial_stock.head()

C:\Users\KristjanLõhmus\AppData\Local\Temp\ipykernel_32036\3296391333.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  initial_stock = sqlio.read_sql_query("SELECT * FROM product", conn)


,barecode,label,category,stock
0,AAA001,Wood Chair,House,10
1,AAA002,Iron bed,House,5
2,AAA003,Mathematics Book,Education,2
3,AAA004,Couch,House,2


##### b. Customer Alice order by phone three products concerning (Wood chair, 5); (Mathematics Book’, 4); (Couch, 1). Her primarily choice was to pay by card but unfortunately the payment was rejected so she said she will make the payment by cash after some days.


In [27]:
cursor.execute("""
-- Insert into Invoice table
INSERT INTO Invoice (DateInv, ModePay, Status) VALUES
    (CURRENT_DATE, 'Card', 'Not Paid');

-- Assuming IdCust = 1 and IdInv = 1
-- Insert into OrderC table
INSERT INTO OrderC (DateOrd, StatusOrd, IdCust, IdInv) VALUES
    (CURRENT_DATE, 'Pending', 1, 1);

-- Assuming IdOrd = 1
-- Insert into OrderLigne table
INSERT INTO OrderLigne (Quantity, StatusOrLig, IdOrd, BareCode) VALUES
                                                                    (5, 'Preparation', 1, 'AAA001'),  -- Wood Chair
                                                                    (4, 'Preparation', 1, 'AAA003'),  -- Mathematics Book
                                                                    (1, 'Preparation', 1, 'AAA004');  -- Couch

-- Update the Invoice for payment by cash
UPDATE Invoice
SET ModePay = 'Cash'
WHERE IdInv = 1;
"""
)

In [28]:
sqlio.read_sql_query("""
SELECT 
    c.Name AS CustomerName,
    c.Phone AS CustomerPhone,
    o.DateOrd AS OrderDate,
    o.StatusOrd AS OrderStatus,
    i.DateInv AS InvoiceDate,
    i.DatePay AS PaymentDate,
    i.ModePay AS PaymentMode,
    i.Status AS PaymentStatus,
    ol.Quantity AS ProductQuantity,
    ol.StatusOrLig AS OrderLineStatus,
    p.BareCode AS ProductCode,
    p.Label AS ProductLabel,
    p.Category AS ProductCategory
FROM 
    Customer c
JOIN 
    OrderC o ON c.IdCust = o.IdCust
JOIN 
    Invoice i ON o.IdInv = i.IdInv
JOIN 
    OrderLigne ol ON o.IdOrd = ol.IdOrd
JOIN 
    Product p ON ol.BareCode = p.BareCode
WHERE 
    c.Name = 'Alice' 
    AND c.Phone = '002736014251';

""", conn).head()

C:\Users\KristjanLõhmus\AppData\Local\Temp\ipykernel_32036\4164947445.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sqlio.read_sql_query("""


,customername,customerphone,orderdate,orderstatus,invoicedate,paymentdate,paymentmode,paymentstatus,productquantity,orderlinestatus,productcode,productlabel,productcategory
0,Alice,002736014251,2024-05-15,Pending,2024-05-15,None,Cash,Not Paid,5,Preparation,AAA001,Wood Chair,House
1,Alice,002736014251,2024-05-15,Pending,2024-05-15,None,Cash,Not Paid,4,Preparation,AAA003,Mathematics Book,Education
2,Alice,002736014251,2024-05-15,Pending,2024-05-15,None,Cash,Not Paid,1,Preparation,AAA004,Couch,House


### B. Automation

#### 1. Implement date field auto-insertion for:


In [29]:
# DateInv
cursor.execute("""
ALTER TABLE Invoice
ALTER COLUMN DateInv SET DEFAULT CURRENT_DATE;
""")

In [30]:
# OrderC
cursor.execute("""
ALTER TABLE OrderC
ALTER COLUMN DateOrd SET DEFAULT CURRENT_DATE;
""")

In [31]:
# DateExp 
cursor.execute("""
ALTER TABLE Delivery
ALTER COLUMN DateExp SET DEFAULT CURRENT_DATE;
""")

In [32]:
# DatePay 
cursor.execute("""
CREATE OR REPLACE FUNCTION update_datepay()
RETURNS TRIGGER AS $$
BEGIN
    IF NEW.Status = 'Paid' AND OLD.Status <> 'Paid' THEN
        NEW.DatePay = CURRENT_DATE;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Create the trigger for the Invoice table
CREATE TRIGGER set_datepay
BEFORE UPDATE ON Invoice
FOR EACH ROW
EXECUTE FUNCTION update_datepay();
""")

#### 2. Analyze the automation that start when the Invoice status passes to Paid and implement them using triggers. Remember that you have the ability to control the timing of trigger firing. Below is one possible automation schema:

![Task b_02](screenshots/b_02.png)

In [45]:
# First let's create one for the scenario where an order gets paid
cursor.execute("""
CREATE OR REPLACE FUNCTION update_orderligne_status()
RETURNS TRIGGER AS $$
DECLARE
    order_line RECORD;
    product RECORD;
BEGIN
    -- Loop through each order associated with the invoice
    FOR order_line IN 
        SELECT * FROM OrderLigne 
        WHERE IdOrd IN (SELECT IdOrd FROM OrderC WHERE IdInv = NEW.IdInv)
    LOOP
        -- Fetch product details
        SELECT * INTO product FROM Product WHERE BareCode = order_line.BareCode;

        IF product.Stock >= order_line.Quantity THEN
            -- Enough stock
            UPDATE Stock
            SET StatusOrLig = 'prepared'
            WHERE IdOrLi = order_line.IdOrLi;
            
            UPDATE Product
            SET stock = stock - order_line.Quantity
            WHERE barecode = order_line.barecode;
        ELSE
            -- Not enough stock
            UPDATE Product
            SET StatusOrLig = 'supply'
            WHERE IdOrLi = order_line.IdOrLi;
        END IF;
    END LOOP;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trigger_update_orderligne_status
AFTER UPDATE OF Status ON Invoice
FOR EACH ROW
WHEN (NEW.Status = 'Paid')
EXECUTE FUNCTION update_orderligne_status();

""")

In [35]:
cursor.execute("""
CREATE OR REPLACE FUNCTION update_delivery_details()
RETURNS TRIGGER AS $$
DECLARE
    delivery_id INT;
BEGIN
    -- Create a new delivery entry
    INSERT INTO Delivery (DateExp) VALUES (CURRENT_DATE) RETURNING IdDel INTO delivery_id;
    -- Update delivery ID for order lines with sufficient stock
    UPDATE OrderLigne
    SET IdDel = delivery_id
    WHERE StatusOrLig = 'Sale done'
    AND IdOrd IN (SELECT IdOrd FROM OrderC WHERE IdInv = NEW.IdInv);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_update_delivery_details
AFTER UPDATE OF StatusOrlig ON OrderLigne
FOR EACH ROW
WHEN (NEW.Status = 'Sale done')
EXECUTE FUNCTION update_delivery_details();

""")

In [38]:
cursor.execute("""
CREATE OR REPLACE FUNCTION update_orderc_status()
RETURNS TRIGGER AS $$
DECLARE
    order_id RECORD;
BEGIN
    IF NEW.Status = 'Paid' THEN
        -- Update order status to 'Validate'
        UPDATE OrderC
        SET StatusOrd = 'Validate'
        WHERE IdInv = NEW.IdInv;

        -- Further update based on order line statuses
        FOR order_id IN 
            SELECT IdOrd FROM OrderC WHERE IdInv = NEW.IdInv
        LOOP
            IF EXISTS (SELECT 1 FROM OrderLigne WHERE IdOrd = order_id AND StatusOrLig = 'Pending for supply') THEN
                UPDATE OrderC
                SET StatusOrd = 'Partial'
                WHERE IdOrd = order_id;
            ELSE
                UPDATE OrderC
                SET StatusOrd = 'Shipped'
                WHERE IdOrd = order_id;
            END IF;
        END LOOP;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;


CREATE TRIGGER trigger_update_orderc_status
AFTER UPDATE OF Status ON Invoice
FOR EACH ROW
WHEN (NEW.Status = 'Paid')
EXECUTE FUNCTION update_orderc_status();
""")

In [39]:
cursor.execute("""
CREATE OR REPLACE FUNCTION update_order_line_status()
RETURNS TRIGGER AS $$
DECLARE
    order_line_id RECORD;
BEGIN
    -- Log a message for order lines with insufficient stock
        FOR order_line_id IN 
            SELECT IdOrLi FROM OrderLigne WHERE barecode = NEW.barecode AND statusOrLig = 'supply'
            IF 
        LOOP
        END LOOP;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_log_supply_chain_messages
AFTER UPDATE OF Stock ON Product
FOR EACH ROW
WHEN (NEW.Stock > OLD.Stock)
EXECUTE FUNCTION update_order_line_status();
""")

#### 3. Let’s assume now that Alice is ready for cash payment. Write an update query to set Invoice status and the mode of payment. 

In [40]:
cursor.execute("""
UPDATE Invoice
SET ModePay = 'Cash', Status = 'Paid'
WHERE IdInv IN (
    SELECT o.IdInv
    FROM OrderC o
    JOIN Customer c ON o.IdCust = c.IdCust
    WHERE c.Name = 'Alice' AND c.Phone = '002736014251'
);
""")

UndefinedTable: relation "supplychainmessages" does not exist
LINE 1: INSERT INTO SupplyChainMessages (OrderLineId, Message)
                    ^
QUERY:  INSERT INTO SupplyChainMessages (OrderLineId, Message)
        SELECT IdOrLi, 'Stock not enough, pending for supply'
        FROM OrderLigne
        WHERE StatusOrLig = 'Pending for supply'
        AND IdOrd IN (SELECT IdOrd FROM OrderC WHERE IdInv = NEW.IdInv)
CONTEXT:  PL/pgSQL function log_supply_chain_messages() line 5 at SQL statement


#### 4. If everything performs exactly, then:

In [ ]:
# OrderStatus to partial

sqlio.read_sql_query("""
SELECT idord, statusord FROM OrderC
""",conn).head()

In [ ]:
# Order ligne status for Wood chair to prepared

sqlio.read_sql_query("""
SELECT statusorlig FROM OrderLigne ol left join Product p on ol.bareCode = p.BareCode
WHERE p.label = 'Wood Chair'
""",conn).head()

In [ ]:
# Order ligne status for Couch to prepared

sqlio.read_sql_query("""
SELECT statusorlig FROM OrderLigne ol left join Product p on ol.bareCode = p.BareCode
WHERE p.label = 'Couch'
""",conn).head()

In [ ]:
# Order ligne status for Mathematics Book to Supply

sqlio.read_sql_query("""
SELECT statusorlig FROM OrderLigne ol left join Product p on ol.bareCode = p.BareCode
WHERE p.label = 'Mathematics Book'
""",conn).head()

In [ ]:
# A tuple in the delivery whose ID is the same as that used in the Id delivery in order ligne  for the ‘wood chair’ and ‘Couh’.
# The Id delivery for product ‘Mahematic book’ is still tonull.

sqlio.read_sql_query("""
SELECT 
    p.Label AS ProductLabel,
    ol.IdDel AS DeliveryId
FROM 
    OrderLigne ol
JOIN 
    Product p ON ol.BareCode = p.BareCode
LEFT JOIN 
    Delivery d ON ol.IdDel = d.IdDel
WHERE 
    (p.Label IN ('Wood Chair', 'Couch') AND ol.IdDel IS NOT NULL)
    OR (p.Label = 'Mathematics Book' AND ol.IdDel IS NULL);

""",conn).head()

In [ ]:
# Stocks of these products also should decrease

new_stock = sqlio.read_sql_query("""
SELECT barecode, label, stock as new_stock from Product
""",conn)
initial_stock.rename(columns={'stock': 'old_stock'}, inplace=True)
pd.merge(initial_stock[['barecode', 'old_stock']], new_stock, on='barecode', how='outer')[['barecode', 'label', 'old_stock','new_stock']]

#### 5.  Write a query to provision the stock of Mathematic book. Let’s say that we add to the stock 10.

In [ ]:
cursor.execute("""
UPDATE Product
SET Stock = Stock + 10
WHERE Label = 'Mathematics Book';
""")

In [ ]:
# Check your database content mainly if you have OrderStatus to Validate
sqlio.read_sql_query("""
SELECT idord, statusord FROM OrderC
""",conn).head()